<a href="https://colab.research.google.com/github/Monferium/APM1210/blob/main/2_Resampling_and_Inferential_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎯 Problem Statement

We aim to answer:

> *“How does phone addiction affect teenagers’ sleep, exercise, self-esteem, and school performance?”*

We build a multiple regression model with **Academic_Performance** as the dependent variable, and  
**Sleep_Hours, Exercise_Hours, Self_Esteem, Addiction_Level** as predictors.



# 📊 TASK 2: Bootstrap and Jackknife Analysis

We investigates the relationship between teenage phone addiction and adolescent well-being, focusing on school performance as the dependent variable.  
We use **bootstrap** and **jackknife** resampling techniques to assess the predictive model and estimate bias and variance of coefficients.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

np.random.seed(1000)

df = pd.read_csv('/content/sample_data/teen_phone_addiction_dataset.csv')
df30 = df.iloc[:30]

df30.head()

,ID,Name,Age,Gender,Location,School_Grade,Daily_Usage_Hours,Sleep_Hours,Academic_Performance,Social_Interactions,...,Screen_Time_Before_Bed,Phone_Checks_Per_Day,Apps_Used_Daily,Time_on_Social_Media,Time_on_Gaming,Time_on_Education,Phone_Usage_Purpose,Family_Communication,Weekend_Usage_Hours,Addiction_Level
0,1,Shannon Francis,13,Female,Hansonfort,9th,4.0,6.1,78,5,...,1.4,86,19,3.6,1.7,1.2,Browsing,4,8.7,10.0
1,2,Scott Rodriguez,17,Female,Theodorefort,7th,5.5,6.5,70,5,...,0.9,96,9,1.1,4.0,1.8,Browsing,2,5.3,10.0
2,3,Adrian Knox,13,Other,Lindseystad,11th,5.8,5.5,93,8,...,0.5,137,8,0.3,1.5,0.4,Education,6,5.7,9.2
3,4,Brittany Hamilton,18,Female,West Anthony,12th,3.1,3.9,78,8,...,1.4,128,7,3.1,1.6,0.8,Social Media,8,3.0,9.8
4,5,Steven Smith,14,Other,Port Lindsaystad,9th,2.5,6.7,56,4,...,1.0,96,20,2.6,0.9,1.1,Gaming,10,3.7,8.6
